In [1]:
import os
import sys
sys.path.append('/home/fruitspec-lab/FruitSpec/Code/fsCounter')
from omegaconf import OmegaConf
import pyzed.sl as sl
import cv2
import numpy as np
from scipy.stats import gaussian_kde
import kornia as K
from tqdm import tqdm
import numpy as np
import collections
import pandas as pd
from vision.misc.help_func import get_repo_dir, scale_dets, validate_output_path, scale
from vision.depth.zed.svo_operations import get_frame, get_depth, get_point_cloud, get_dimensions, sl_get_dimensions

repo_dir = get_repo_dir()
sys.path.append(os.path.join(repo_dir, 'vision', 'detector', 'yolo_x'))

from vision.pipelines.detection_flow import counter_detection
from vision.pipelines.misc.filters import filter_by_distance, filter_by_size, filter_by_height, sort_out
from vision.tracker.fsTracker.score_func import compute_dist_on_vec
from vision.data.results_collector import ResultsCollector
from vision.tools.translation import translation as T
from vision.tools.camera import is_sturated
from vision.tools.color import get_hue
from vision.tools.video_wrapper import video_wrapper
from vision.tools.image_stitching import plot_2_imgs
import matplotlib.pyplot as plt
import seaborn as sns
import kornia as K


In [2]:
repo_dir = get_repo_dir()
pipeline_config = "/home/fruitspec-lab/FruitSpec/Code/fsCounter/vision/pipelines/config/pipeline_config.yaml"
runtime_config = "/home/fruitspec-lab/FruitSpec/Code/fsCounter/vision/pipelines/config/runtime_config.yaml"
cfg = OmegaConf.load(pipeline_config)
args = OmegaConf.load(runtime_config)

validate_output_path(args.output_folder)

In [3]:
detector = counter_detection(cfg, args)
results_collector = ResultsCollector(rotate=args.rotate)
translation = T(cfg.translation.translation_size, cfg.translation.dets_only, cfg.translation.mode)

cam = video_wrapper(args.movie_path, args.rotate, args.depth_minimum, args.depth_maximum)

# Read until video is completed
print(f'Inferencing on {args.movie_path}\n')
number_of_frames = cam.get_number_of_frames()


loading checkpoint from /home/fruitspec-lab/FruitSpec/weights/Run_6_Dec_2022_1Class_aug_tasqV2/best_ckpt.pth
loaded checkpoint done.
Inferencing on /home/fruitspec-lab/Downloads/0.8m1sts/ZED_1.svo



In [4]:
measures_path = "/home/fruitspec-lab/Downloads/0.8m1sts/output/measures.csv"
measures_frame = pd.read_csv(measures_path)

In [5]:
def plot_image_grid(images, nrows, ncols, titles=None, figsize=None, xlabels=None, ylabels=None, cmap='viridis'):
    """
    Plots a grid of images using matplotlib.

    Parameters:
    images (list): A list of numpy arrays representing the images to be plotted.
    nrows (int): The number of rows in the grid.
    ncols (int): The number of columns in the grid.
    titles (list, optional): A list of strings representing the titles of the images. Must have the same length as images.
    figsize (tuple, optional): A tuple representing the size of the figure. Defaults to (ncols * 5, nrows * 5).
    xlabels (list, optional): A list of strings representing the x-axis labels for each image. Must have the same length as images.
    ylabels (list, optional): A list of strings representing the y-axis labels for each image. Must have the same length as images.
    cmap (str, optional): The color map to use when plotting the images. Defaults to 'viridis'.

    Returns:
    None
    """
    n_images = len(images)
    if isinstance(figsize, type(None)):
        figsize = (ncols * 5, nrows * 5)
    titles, xlabels, ylabels = (np.full(n_images, "") if isinstance(titles, type(None)) else arr
                                for arr in [titles, xlabels, ylabels])
    fig, axs = plt.subplots(nrows, ncols, figsize=figsize)
    for i, ax in enumerate(axs.flat):
        image = images[i]
        ax.imshow(image)
        ax.set_title(titles[i])
        ax.set_xlabel(xlabels[i])
        ax.set_ylabel(ylabels[i])
    plt.tight_layout()
    plt.show()

def get_real_world_dims_with_correction(depth_map, fx = 1065.98388671875, fy = 1065.98388671875, resized = True):
    """
    calculates each pixel size based on trigo
    :param depth_map: distance_map to each point or an empty string
    :return: size for each pixel
    """
    pic_size = depth_map.shape
    if resized:
        resize_fator_x = 1080/pic_size[0]
        resize_fator_y = 1920/pic_size[1]
    else:
        resize_fator_x = 1
        resize_fator_y = 1
    x0 = pic_size[1] /2
    y0 = pic_size[0] /2
    pixel_mm = 0.002
    focal_len = (fx + fy) / 2 * pixel_mm
    x_range = np.arange(1, pic_size[1]+1)
    X_pix_dist_from_center = np.abs(np.array([x_range for i in range(pic_size[0])]) - x0)
    X_mm_dist_from_center = (X_pix_dist_from_center * (X_pix_dist_from_center+1)*(pixel_mm**2))
    beta = np.arctan(0.001/(focal_len + (X_mm_dist_from_center/focal_len)))
    gamma = np.arctan((X_pix_dist_from_center+1)*pixel_mm/focal_len)
    size_x = (np.tan(gamma) - np.tan(gamma-beta))*depth_map*2*resize_fator_x
    size_y = (np.tan(gamma) - np.tan(gamma-beta))*depth_map*2*resize_fator_y
    return size_x, size_y, size_x*size_y

def get_pix_size(depth, box, fx = 1065.98388671875, fy = 1065.98388671875,
                 pixel_mm = 0.0002, org_size = np.array([1920,1080])):
    """
    Calculates the size of a pixel in millimeters given a distance from the camera and the intrinsic parameters of the camera.

    Args:
        depth (float): The depth from the camera to the object in meters.
        box (list): ROI for pixel size int hte following format: x1,y1,x2,y2.
        fx (float): The focal length of the camera in the x direction in pixels. Default is 1065.98388671875.
        fy (float): The focal length of the camera in the y direction in pixels. Default is 1065.98388671875.
        pixel_mm (float): The size of a pixel in millimeters. Default is 0.002.
        org_size (ndarray): The size of the image in pixels. Default is np.array([1920, 1080]).

    Returns:
        size_pix (float): The size of a pixel
    """
    x1,y1,x2,y2 = box
    y0, x0 = org_size/2
    focal_len = (fx + fy) / 2 * pixel_mm
    x_range = np.arange(x1, x2+1)
    x_pix_dist_from_center = np.abs(np.array([x_range for i in range(y2-y1)]) - x0)
    x_mm_dist_from_center = (x_pix_dist_from_center * (x_pix_dist_from_center+1)*(pixel_mm**2))
    beta = np.arctan(0.001/(focal_len + (x_mm_dist_from_center/focal_len)))
    gamma = np.arctan((x_mm_dist_from_center+1)*pixel_mm/focal_len)
    size_pix = (np.tan(gamma) - np.tan(gamma-beta))*depth*2
    return size_pix


def cut_center_of_box(image, margin=0.05):
    """
    Cuts the center of the box.
    
    Args:
    - image: A 3D Numpy array representing a cropped xyz image.
    - margin: A float representing the percentage of margin to remove from the center of the image.

    Returns:
    - A 3D Numpy array representing the cropped image with the outside of the box removed.
    """
    t, l, (b, r) = 0 ,0, image.shape[:2]
    y_max, x_max = image.shape[:2]
    h_m = int((b-t)*margin)
    w_m = int((r-l)*margin)
    cut_box = image[max(0, t+h_m):min(y_max, b-h_m), max(0, l+w_m):min(x_max, r-w_m)]
    return cut_box


def xyz_center_of_box(image, method="median"):
    """
    Calculates the median or mean x, y, and z coordinates of the fruit.

    Args:
    - image: A 3D Numpy array representing a cropped xyz image.
    - method: A string representing the method to use to calculate the center of the fruit. Must be either "median" or "mean".

    Returns:
    - A tuple of floats representing the x, y, and z coordinates of the center of the fruit.
    """
    if method=="median":
        cut_box = cut_center_of_box(image, 0.025)
        x_median = np.nanmedian(cut_box[:, :, 0])
        y_median = np.nanmedian(cut_box[:, :, 1])
        z_median = np.nanmedian(cut_box[:, :, 2])
    elif method=="mean":
        cut_box = cut_center_of_box(image, 0.4)
        x_median = np.nanmean(cut_box[:, :, 0])
        y_median = np.nanmean(cut_box[:, :, 1])
        z_median = np.nanmean(cut_box[:, :, 2])
    else: # calculates only on the edge of the cut box
        cut_box = cut_center_of_box(image, 0.25).copy()
        if cut_box.shape[0] > 10 and cut_box.shape[1] > 10:
            cut_box[5:-5,5:-5] = np.nan
        x_median = np.nanmedian(cut_box[:, :, 0])
        y_median = np.nanmedian(cut_box[:, :, 1])
        z_median = np.nanmedian(cut_box[:, :, 2])
    return x_median, y_median, z_median


def dist_to_box_center(image, method="median"):
    """
    Calculates the distance from the camera to the center of the fruit.

    Args:
    - image: A 3D Numpy array representing a cropped xyz image.
    - method: A string representing the method to use to calculate the center of the fruit. Must be either "median" or "mean".

    Returns:
    - A float representing the distance from the camera to the center of the fruit.
    """
    return np.sum(np.array(list(xyz_center_of_box(image, method)))**2)

def depth_to_box_center(image, method="median"):
    """
    Calculates the depth from the camera to the center of the fruit.

    Args:
    - image: A 3D Numpy array representing a cropped xyz image.
    - method: A string representing the method to use to calculate the center of the fruit. Must be either "median" or "mean".

    Returns:
    - A float representing the depth from the camera to the center of the fruit.
    """
    return xyz_center_of_box(image, method)[2]

def get_dims_w_pixel_size(pc_img, box, center_method="median"):
    """
    Calculates the width and height of a 2D bounding box in millimeters, based on the pixel size of the image.

    Args:
    - pc_img: A 3D Numpy array representing a point cloud image.
    - box: A tuple of integers representing the (x1, y1, x2, y2) coordinates of the bounding box.
    - center_method: A string representing the method to use to calculate the center of the fruit. Must be either "median" or "mean".

    Returns:
    - A tuple of floats representing the width and height of the bounding box in millimeters.
    """
    dist = depth_to_box_center(pc_img, center_method)
    size_pix = get_pix_size(dist, box)
    width = np.mean(np.sum(size_pix,axis = 0))
    height = np.mean(np.sum(size_pix,axis = 1))
    return width, height 


In [6]:
def get_track_id_frames(measures_frame, track_id):
    """
    Returns a numpy array of frames associated with a given track ID.

    Parameters:
    measures_frame (pandas.DataFrame): A dataframe containing measurements data.
    track_id (int): The ID of the track to retrieve frames for.

    Returns:
    numpy.ndarray: A 1-dimensional array of frames associated with the given track ID.
    """
    return measures_frame[measures_frame["track_id"] == track_id]["frame"].values

def get_track_id_boxes(measures_frame, track_id):
    """
    Returns a numpy array of bounding boxes associated with a given track ID.

    Parameters:
    measures_frame (pandas.DataFrame): A dataframe containing measurements data.
    track_id (int): The ID of the track to retrieve bounding boxes for.

    Returns:
    numpy.ndarray: A 2-dimensional array of bounding boxes associated with the given track ID.
        Each row contains the x1, y1, x2, y2 coordinates of a bounding box in the format [x1, y1, x2, y2].
    """
    return measures_frame[measures_frame["track_id"] == track_id][["x1", "y1", "x2", "y2"]].values

def validate_bbox(crop, rgb_img):
    """
    Validates the given bounding box coordinates and ensures that they fall within the dimensions of the RGB image.

    Parameters:
    crop (tuple): A tuple containing the coordinates of the bounding box in the format (x1, y1, x2, y2).
        x1 and y1 are the coordinates of the top-left corner of the bounding box, and x2 and y2 are the coordinates of the bottom-right corner.
    rgb_img (numpy.ndarray): A numpy array representing the RGB image.

    Returns:
    tuple: A tuple containing the validated bounding box coordinates in the format (x1, y1, x2, y2).
        The returned coordinates ensure that the bounding box falls entirely within the dimensions of the RGB image.
    """
    x1, y1, x2, y2 = crop
    x1 = max(x1, 0)
    y1 = max(y1, 0)
    h, w = rgb_img.shape[:2]
    x2 = min(x2, w-1)
    y2 = min(y2, h-1)
    return x1, y1, x2, y2


def get_track_id_images(measures_frame, track_id, args):
    cam = video_wrapper(args.movie_path, args.rotate, args.depth_minimum, args.depth_maximum)
    frame_numbers = get_track_id_frames(measures_frame, track_id)
    boxes = get_track_id_boxes(measures_frame, track_id)
    rgb_images, pc_images = [], []
    for i in tqdm(range(len(frame_numbers))):
        frame, crop = frame_numbers[i], boxes[i]
        rgb_img, _, pc_img = cam.get_zed(frame)
        x1,y1,x2,y2 = validate_bbox(crop, rgb_img)
        rgb_images.append(rgb_img[y1:y2,x1:x2])
        pc_images.append(pc_img[y1:y2,x1:x2])
    cam.close()
    return rgb_images, pc_images

def kde_filtering(centers, thresh=0.5):
    """
    Applies a Kernel Density Estimation (KDE) filtering on a set of 3D points.

    Args:
        centers (np.ndarray): A numpy array of shape (n, 3) representing the 3D coordinates of the points to filter.
        thresh (float): A threshold value to filter out points with low density. Default is 0.5.

    Returns:
        np.ndarray: A numpy array of shape (n, 3) where each filtered 3D points is replaced with np.nan
    """
    finite_logical = np.all(np.isfinite(centers), axis=1)
    if not sum(finite_logical):
        return centers
    finite_centers = centers[finite_logical].copy()
    kernel = gaussian_kde(finite_centers.T)
    points_density = np.full(len(centers), np.nan)
    points_density[finite_logical] = kernel(finite_centers.T)
    points_density = points_density/np.nansum(points_density)
    filtered_center = centers.copy()
    filtered_center[points_density <thresh/ len(finite_logical)] = np.nan
    return filtered_center

In [7]:
# rgb_images, pc_images = get_track_id_images(measures_frame, 7)
# plot_2_imgs(rgb_images[0], pc_images[0][:,:,2])

In [8]:
# for track_id in [6,7,8,28]:
#     print(track_id)
#     frame_numbers = get_track_id_frames(measures_frame, track_id)
#     boxes = get_track_id_boxes(measures_frame, track_id)
#     rgb_images, pc_images = [], []
#     for frame, crop in tqdm(zip(frame_numbers, boxes)):
#         rgb_img, _, pc_img = cam.get_zed(frame)
#         x1,y1,x2,y2 = validate_bbox(crop, rgb_img)
#         rgb_images.append(rgb_img[y1:y2,x1:x2])
#         pc_images.append(pc_img[y1:y2,x1:x2])
#     plot_image_grid(rgb_images,5,5)

In [9]:
def filter_xyz_outliers(crop, nstd=1.5, as_points=True):
    """
    Filters out the outliers from the 3D points in the given crop.

    Args:
        crop (ndarray): A numpy array of shape (height, width, 3) containing the 3D points.
        nstd (float): The number of standard deviations to consider for defining the range of valid values.
        as_points (bool): Whether to return the filtered 3D points as an array of points or as an array of the same shape
            as the input crop.

    Returns:
        ndarray: A numpy array of filtered 3D points. If as_points is True, this is a numpy array of shape (n, 3),
            where n is the number of valid 3D points. Otherwise, it is a numpy array of the same shape as the input crop.
    """
    centers = crop.reshape(-1, 3)
    filtered_center = centers.copy()
    for channel in [0, 1, 2]:
        channel_vals = centers[:, channel]
        max_val = np.abs(np.nanmedian(channel_vals) + nstd * np.nanstd(channel_vals))
        filtered_center[np.abs(channel_vals) > max_val] = np.nan
    if as_points:
        return filtered_center
    return filtered_center.reshape(crop.shape)


def ellipsoid_fit(filtered_center):
    """
    Fits an ellipsoid to a set of 3D points using least squares estimation.

    Args:
    - filtered_center (numpy array): An N x 3 array of N 3D points in the form (x, y, z).

    Returns:
    - radius (float): The radius of the fitted ellipsoid.
    - semi_axis_1 (float): The length of the semi-major axis of the ellipsoid.
    - semi_axis_2 (float): The length of the semi-intermediate axis of the ellipsoid.
    - semi_axis_3 (float): The length of the semi-minor axis of the ellipsoid.
    """
    filtered_center[np.abs(filtered_center) > 2] = np.nan
    A = np.column_stack([filtered_center, np.ones(len(filtered_center))])
    good_rows = np.all(np.isfinite(A), axis=1)
    A = A[good_rows]

    #   Assemble the f matrix
    f = np.zeros((len(A),1))
    f[:,0] = np.sum(A[:,:3]**2, axis = 1)
    C, residules, rank, singval = np.linalg.lstsq(A,f)
    C = np.abs(C)
    #   solve for the radius
    t = (C[0]*C[0])+(C[1]*C[1])+(C[2]*C[2])+C[3]
    radius = np.sqrt(t)
    # channels are switched
    return radius, np.sqrt(C[1]), np.sqrt(C[0]), np.sqrt(C[2])

In [10]:
from sklearn.linear_model import RANSACRegressor
def ellipsoid_fit_ransac(filtered_center, num_iterations=100, inlier_threshold=0.1):
    """
    Fits an ellipsoid to a set of 3D points using RANSAC.

    Args:
    - filtered_center (numpy array): An N x 3 array of N 3D points in the form (x, y, z).
    - num_iterations (int): The number of iterations to run RANSAC for.
    - inlier_threshold (float): The maximum distance from a point to the fitted ellipsoid to be considered an inlier.

    Returns:
    - radius (float): The radius of the fitted ellipsoid.
    - semi_axis_1 (float): The length of the semi-major axis of the ellipsoid.
    - semi_axis_2 (float): The length of the semi-intermediate axis of the ellipsoid.
    - semi_axis_3 (float): The length of the semi-minor axis of the ellipsoid.
    """
    filtered_center[np.abs(filtered_center) > 2] = np.nan
    A = np.column_stack([filtered_center, np.ones(len(filtered_center))])
    good_rows = np.all(np.isfinite(A), axis=1)
    A = A[good_rows]
    if len(A) < 5:
        return np.nan, np.nan, np.nan, np.nan
    f = np.zeros((len(A),1))
    f[:,0] = np.sum(A[:,:3]**2, axis = 1)

    model = RANSACRegressor(estimator=None, residual_threshold=inlier_threshold, max_trials=num_iterations)
    model.fit(A,f)

    # Extract the coefficients of the fitted ellipsoid
    coef = model.estimator_.coef_[0]
    C = np.abs(np.concatenate([coef[:3], [1], coef[3:]]))

    #   solve for the radius
    t = (C[0]*C[0])+(C[1]*C[1])+(C[2]*C[2])+C[3]
    radius = np.sqrt(t)

    # channels are switched
    return radius, np.sqrt(C[1]), np.sqrt(C[0]), np.sqrt(C[2]) 

In [11]:
for track_id in [6,7,12,28]:
    cam = video_wrapper(args.movie_path, args.rotate, args.depth_minimum, args.depth_maximum)
    frame_numbers = get_track_id_frames(measures_frame, track_id)
    boxes = get_track_id_boxes(measures_frame, track_id)
    ellps_w, ellps_h, ellps_z = [], [], []
    pix_size_w_mean, pix_size_h_mean = [], []
    pix_size_w_med, pix_size_h_med = [], []
    for i in tqdm(range(len(frame_numbers))):
        frame, box = frame_numbers[i], boxes[i]
        rgb_img, _, pc_img = cam.get_zed(frame)
        x1,y1,x2,y2 = validate_bbox(box, rgb_img)
        pc_crop = pc_img[y1:y2, x1:x2,:3]
#         filtered_center = kde_filtering(pc_crop[:,:,:3].reshape(-1,3))
#         _, width, height, z_w = ellipsoid_fit_ransac(filtered_center)
#         if width > 200 or height > 200 or z_w > 200: #if fruits are too big its probably due to noise
#             width, height, z_w = np.nan, np.nan, np.nan
        ellps_w.append(width), ellps_h.append(height), ellps_z.append(z_w)
        width, height = get_dims_w_pixel_size(pc_img, [x1,y1,x2,y2], "mean")
        pix_size_w_mean.append(width), pix_size_h_mean.append(height)
        width, height = get_dims_w_pixel_size(pc_img, [x1,y1,x2,y2], "median")
        pix_size_w_med.append(width), pix_size_h_med.append(height)

    plt.imshow(rgb_img[y1:y2, x1:x2])
    plt.title(track_id)
    plt.show()
    print(np.nanmean(ellps_w), np.nanstd(ellps_w), np.nanmean(ellps_h), np.nanstd(ellps_h), np.nanmean(ellps_z), np.nanstd(ellps_z)
     ,np.nanmean(pix_size_w_mean), np.nanstd(pix_size_w_mean),np.nanmean(pix_size_h_mean), np.nanstd(pix_size_h_mean)
    ,np.nanmean(pix_size_w_med), np.nanstd(pix_size_w_med),np.nanmean(pix_size_h_med), np.nanstd(pix_size_h_med))

  0%|                                                                                                                                                                                                   | 0/107 [00:00<?, ?it/s]


NameError: name 'width' is not defined

In [ ]:
track_id = 5
cam = video_wrapper(args.movie_path, args.rotate, args.depth_minimum, args.depth_maximum)
frame_numbers = get_track_id_frames(measures_frame, track_id)
boxes = get_track_id_boxes(measures_frame, track_id)
ellps_w, ellps_h, ellps_z = [], [], []
pix_size_w_mean, pix_size_h_mean = [], []
pix_size_w_med, pix_size_h_med = [], []
i = len(frame_numbers) -1
frame, box = frame_numbers[i], boxes[i]
rgb_img, _, pc_img = cam.get_zed(frame)
x1,y1,x2,y2 = validate_bbox(box, rgb_img)
pc_crop = pc_img[y1:y2, x1:x2,:3]
plt.imshow(rgb_img[y1:y2, x1:x2])
plt.title(track_id)
plt.show()

In [ ]:
cut_box_pc = cut_center_of_box(pc_crop, 0.1)
cut_box_rgb = cut_center_of_box(rgb_img[y1:y2, x1:x2], 0.1)
plot_2_imgs(cut_box_pc[:, :, 2], cut_box_rgb)

In [ ]:
def apply_sobol(det_crop, plot_change=False):
    """
    applies sobol filterning on image
    :param det_crop: image to apply filter on
    :param plot_change: flag to show the image after applying sobol
    :return: image after sobol filtering
    """
    torch_img = K.utils.image_to_tensor(det_crop)
    torch_img = torch_img[None, ...].float() / 255.
    torch_img = K.enhance.adjust_contrast(torch_img, 0.5)
    torch_img_gray = K.color.rgb_to_grayscale(torch_img)
    processed_img = K.filters.sobel(torch_img_gray, True, 1e-3)  # BxCx2xHxW
    if plot_change:
        plot_2_imgs(det_crop, processed_img.detach().numpy()[0, 0] > 0.05)
    return processed_img
apply_sobol(rgb_img[y1:y2, x1:x2], True)

In [ ]:
stop

In [ ]:
rgb_images, pc_images = get_track_id_images(measures_frame, 5, args)

In [ ]:
w = []
h = []
for track_id in range(len(pc_images)):
    pc_img = pc_images[track_id]
    box = get_track_id_boxes(measures_frame, 5)[track_id]
    wid,hei = get_dims_w_pixel_size(pc_img, box, "mean")
    w.append(wid), h.append(hei)
np.mean(w), np.mean(h), np.std(w), np.std(h)

In [ ]:
pc_images[-1][30,:,0]

In [ ]:
pc_images[-1][30,:,1]

In [ ]:
arr = pc_images[-1][30,:,2].flatten()
sns.kdeplot(arr)
plt.vlines(np.nanmedian(arr), 0,17.5,color = "black")
plt.vlines(np.nanmedian(arr) + 2*arr.std(),0,17.5,color = "green")

In [ ]:
centers = pc_images[-1][:,:,:3].reshape(-1,3)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection='3d')
ax.scatter3D(-centers[:, 2], -centers[:, 0], -centers[:, 1])
# for i, label in enumerate(fruit_3d_space.keys()):
#     ax.text(-centers[i, 2], -centers[i, 0], -centers[i, 1], '%s' % (str(label)), size=10, zorder=1,
#             color='k')
ax.set_xlabel('z')
ax.set_ylabel('x')
ax.set_zlabel('y')
ax.view_init(0,15)
plt.show()

In [ ]:
import numpy as np
from scipy import stats
def measure(n):
    "Measurement model, return two coupled measurements."
    m1 = np.random.normal(size=n)
    m2 = np.random.normal(scale=0.5, size=n)
    return m1+m2, m1-m2
m1, m2 = measure(2000)
xmin = m1.min()
xmax = m1.max()
ymin = m2.min()
ymax = m2.max()
X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([X.ravel(), Y.ravel()])
values = np.vstack([m1, m2])
values.shape

In [ ]:
values

In [ ]:
finite_centers = centers[np.all(np.isfinite(centers),axis = 1)]
kernel = gaussian_kde(finite_centers.T)

In [ ]:
plt.hist(kernel(finite_centers.T)/np.sum(kernel(finite_centers.T)))

In [ ]:
0.0001*len(finite_centers)

In [ ]:
for i in range(len(rgb_images)):
    rgb_img = rgb_images[i]
    # plt.imshow(rgb_img)
    centers = pc_images[i][:,:,:3].reshape(-1,3)
    filtered_center = kde_filtering(centers)
    # max_z = min(np.abs(np.nanmedian(centers[:,2]) + 2 * np.nanstd(centers[:,2])),1.1)
    # filtered_center[np.abs(centers[:,2]) > max_z] = np.nan
    # filtered_center[np.abs(centers[:,1]) > np.abs(np.nanmedian(centers[:,1]) + 2 * np.nanstd(centers[:,1]))] = np.nan
    # filtered_center[np.abs(centers[:,0]) > np.abs(np.nanmedian(centers[:,0]) + 2* np.nanstd(centers[:,0]))] = np.nan
    filtere_rgb = rgb_img.copy()
    filtere_rgb[np.isnan(filtered_center.reshape(rgb_img.shape))] = 0
    plot_2_imgs(rgb_img, filtere_rgb)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
rgb_img = rgb_images[-4]
# plt.imshow(rgb_img)
centers = pc_images[-4][:,:,:3].reshape(-1,3)
finite_centers = centers[np.all(np.isfinite(centers),axis = 1)].copy()
kernel = gaussian_kde(finite_centers.T)
points_density = np.array([])
for point in centers:
    if not np.sum(np.isnan(point)):
        points_density = np.append(points_density, kernel(point))
    else:
        points_density = np.append(points_density, np.nan)

points_density = points_density/np.nansum(points_density)
filtered_center = finite_centers.copy()
filtered_center[points_density <0.5/ len(finite_centers)] = np.nan
# max_z = min(np.abs(np.nanmedian(centers[:,2]) + 2 * np.nanstd(centers[:,2])),1.1)
# filtered_center[np.abs(centers[:,2]) > max_z] = np.nan
# filtered_center[np.abs(centers[:,1]) > np.abs(np.nanmedian(centers[:,1]) + 2 * np.nanstd(centers[:,1]))] = np.nan
# filtered_center[np.abs(centers[:,0]) > np.abs(np.nanmedian(centers[:,0]) + 2* np.nanstd(centers[:,0]))] = np.nan
filtere_rgb = rgb_img.copy()
filtere_rgb[np.isnan(filtered_center.reshape(rgb_img.shape))] = 0
plot_2_imgs(rgb_img, filtere_rgb)
def plot_3d_scatter(angle1, angle2, angle3):
    fig = plt.figure(figsize=(10, 10))
    ax = plt.axes(projection='3d')
    ax.scatter3D(-filtered_center[:, 2], -filtered_center[:, 1], -filtered_center[:, 0],s = 1)
    ax.set_xlabel('z')
    ax.set_ylabel('x')
    ax.set_zlabel('y')
    ax.view_init(angle1, angle2, angle3)
    plt.show()
    
# define the sliders for the viewing angles
angle1_slider = widgets.FloatSlider(min=-180, max=180, step=1, value=0)
angle2_slider = widgets.FloatSlider(min=-180, max=180, step=1, value=0)
angle3_slider = widgets.FloatSlider(min=-180, max=180, step=1, value=0)

# create the interactive plot
interactive_plot = interact(plot_3d_scatter, angle1=angle1_slider, angle2=angle2_slider, angle3 = angle3_slider)

# display the interactive plot
display(interactive_plot)


In [ ]:
import math
def ellipsoid_fit(filtered_center):
    spX=filtered_center[:,0]
    spY=filtered_center[:,1]
    spZ=filtered_center[:,2]
    falut_rows = np.any(np.isnan(np.column_stack([spX,spY,spZ])), axis=1)
    spX,spY,spZ = spX[~falut_rows],spY[~falut_rows],spZ[~falut_rows]

    A = np.zeros((len(spX),4))
    A[:,0] = spX*2
    A[:,1] = spY*2
    A[:,2] = spZ*2
    A[:,3] = 1

    #   Assemble the f matrix
    f = np.zeros((len(spX),1))
    f[:,0] = (spX*spX) + (spY*spY) + (spZ*spZ)
    C, residules, rank, singval = np.linalg.lstsq(A,f)
    C = np.abs(C)
    #   solve for the radius
    t = (C[0]*C[0])+(C[1]*C[1])+(C[2]*C[2])+C[3]
    radius = np.sqrt(t)
    # channels are switched
    return radius, np.sqrt(C[1]), np.sqrt(C[0]), np.sqrt(C[2])

rgb_img = rgb_images[-1]
plt.imshow(rgb_img)
plt.show()
r = []
w = []
h = []
z = []
for i in range(len(pc_images)):
    centers = pc_images[i][:,:,:3].reshape(-1,3)
    filtered_center = centers.copy()
    filtered_center[np.abs(centers[:,2]) > np.abs(np.nanmedian(centers[:,2]) + 2 * np.nanstd(centers[:,2]))] = np.nan
    filtered_center[np.abs(centers[:,1]) > np.abs(np.nanmedian(centers[:,1]) + 2 * np.nanstd(centers[:,1]))] = np.nan
    filtered_center[np.abs(centers[:,0]) > np.abs(np.nanmedian(centers[:,0]) + 2* np.nanstd(centers[:,0]))] = np.nan
    r_t, w_t, h_t, z_t = ellipsoid_fit(filtered_center)
    r.append(r_t)
    w.append(w_t)
    h.append(h_t)
    z.append(z_t)
np.mean(r), np.mean(w),np.mean(h),np.mean(z)

In [ ]:
np.std(r), np.std(w),np.std(h)

In [ ]:
x,y,z = filtered_center[:,1][~falut_rows],filtered_center[:,0][~falut_rows],filtered_center[:,2][~falut_rows]
A = np.array([x**2, y**2, z**2]).T

# vector of ones
O = np.ones(len(x))

# least squares solver
B, resids, rank, s = np.linalg.lstsq(A, O)

# solving for a, b, c
a_ls = np.sqrt(1.0/B[0])
b_ls = np.sqrt(1.0/B[1])
c_ls = np.sqrt(1.0/B[2])
a_ls, b_ls, c_ls